# Step 5: Measure Population Fidelity (PF)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from math import sqrt
import sys

sys.path.append('../src')

from PF_metrics import *
from utils import *

In [2]:
config = getExperimentConfig()
folders = config['folders']
settings = getPicklesFromDir(config['folders']['settings_dir'])
display(settings)

[{'meta': {'name': 'Diabetes',
   'id': 'D0',
   'filename': 'diabetes.csv',
   'target': 'Outcome',
   'ordinal_features': None,
   'numeric_features': ['DiabetesPedigreeFunction',
    'BMI',
    'Insulin',
    'Glucose',
    'Age',
    'SkinThickness',
    'BloodPressure',
    'Pregnancies'],
   'text_features': None,
   'categorical_features': None,
   'meta_data': {'fields': {'Pregnancies': {'type': 'numerical',
      'subtype': 'integer',
      'transformer': 'FloatFormatter'},
     'Glucose': {'type': 'numerical',
      'subtype': 'integer',
      'transformer': 'FloatFormatter'},
     'BloodPressure': {'type': 'numerical',
      'subtype': 'integer',
      'transformer': 'FloatFormatter'},
     'SkinThickness': {'type': 'numerical',
      'subtype': 'integer',
      'transformer': 'FloatFormatter'},
     'Insulin': {'type': 'numerical',
      'subtype': 'integer',
      'transformer': 'FloatFormatter'},
     'BMI': {'type': 'numerical',
      'subtype': 'float',
      'transform

In [3]:
original_data = pd.read_csv("../data/real/diabetes.csv")
sd0q1 = pd.read_csv("../data/synthetic/SD0Q1_0.csv")
sd0q2 = pd.read_csv("../data/synthetic/SD0Q2_0.csv")
#display(sd0q1)
#display(sd0q2)

In [4]:
sd0q1.drop(columns='Unnamed: 0', inplace=True)
#display(sd0q1.head())
sd0q2.drop(columns='Unnamed: 0', inplace=True)
#display(sd0q2.head())

In [5]:
pmse, pmse_time = pmse(original_data=original_data, synthetic_data=sd0q1)

print("pmse: ", pmse)
print("pmse time: ", pmse_time)

pmse:  0.16754260446744224
pmse time:  0.025068599999999996


In [7]:
from math import log
import pandas as pd

n_clusters = 70
metadata = settings[0]['meta']['meta_data']

c1, _ = cluster_metric(original_data=original_data, 
                             synthetic_data=sd0q1, 
                             num_clusters=n_clusters, 
                             metadata=metadata)

print(f"Cluster analysis metric, SD0Q1: {c1}, Log: {log(c1)}" )

Cluster analysis metric, SD0Q1: 3.8109619653246534, Log: 1.3378816416069828


In [8]:
 
    


def olc_compute_all_pf_measures(original_data:pd.DataFrame, synthetic_data:pd.DataFrame, metadata:dict, SD_id:str) -> pd.DataFrame:
    
    # get number of clusters, using the combined number of samples in the synthetic & original data, 
    # and round it to an integer
    one_percent = 0.01
    five_percent = 0.05
    ten_percent = 0.1
    
    k_1  = round( (original_data.shape[0] + synthetic_data.shape[0]) * one_percent)
    k_5  = round( (original_data.shape[0] + synthetic_data.shape[0]) * five_percent)
    k_10 = round( (original_data.shape[0] + synthetic_data.shape[0]) * ten_percent)

    
    measures = {
        'DatasetName': SD_id,
        
        'pMSE': pmse(original_data=original_data, synthetic_data=synthetic_data),
        
        'SpMSE': s_pmse(original_data=original_data, synthetic_data=synthetic_data),
        
        'Cluster_1': cluster_metric(original_data=original_data, 
                                    synthetic_data=synthetic_data, 
                                    num_clusters=k_1, 
                                    metadata=metadata),   
        # TODO: fix 5 & 10 percent
        #'Cluster_5': cluster_metric(original_data=original_data, 
        #                            synthetic_data=synthetic_data, 
        #                            num_clusters=k_5, 
        #                            metadata=metadata),  
        
        #'Cluster_10': cluster_metric(original_data=original_data, 
        #                            synthetic_data=synthetic_data, 
        #                            num_clusters=k_10, 
        #                            metadata=metadata), 
        
        'BNLikelihood': BNLikelihood_metric(original_data=original_data, 
                                            synthetic_data=synthetic_data, 
                                            metadata=metadata),
        
        'BNLogLikelihood': BNLogLikelihood_metric(original_data=original_data, 
                                                  synthetic_data=synthetic_data, 
                                                  metadata=metadata),
        
        'GMLogLikelihood': GMLogLikelihood_metric(original_data=original_data, 
                                                  synthetic_data=synthetic_data, 
                                                  metadata=metadata),

        'ContinousKLDivergence': ContinousKLDivergence_metric(original_data, synthetic_data, metadata),
        
        'DiscreteKLDivergence': DiscreteKLDivergence_metric(original_data, synthetic_data, metadata),
        
        'KSComplement': KSComplement_metric(original_data, synthetic_data, metadata),
        
        'CSTest': CSTest_metric(original_data, synthetic_data, metadata),
        
        'CrossClassification': CrossClassification_metric(original_data, synthetic_data, metadata)
    }
    
    results_df = pd.DataFrame(data=measures, index=[0])
    return results_df

    

In [11]:
pf_measure_columns = [
    'DatasetName',
    'pMSE',
    'pMSE_time',
    's_pMSE',
    's_pMSE_time',
    'Cluster_1',   # num of clusters = 1% of dataset_size
    'Cluster_1_time',   # num of clusters = 1% of dataset_size
    #'Cluster_5',     # num of clusters = 5% of dataset_size
    #'Cluster_10',    # num of clusters = 10% of dataset_size
    'BNLogLikelihood',
    'BNLogLikelihood_time',
    'GMLogLikelihood',
    'GMLogLikelihood_time',
    'ContinousKLDivergence',
    'ContinousKLDivergence_time',
    'DiscreteKLDivergence',
    'DiscreteKLDivergence_time',
    'KSComplement',
    'KSComplement_time',
    'CSTest',
    'CSTest_time',
    'CrossClassification', #Cross-classification 
    'CrossClassification_time', #Cross-classification 
]

result_df = pd.DataFrame(columns=pf_measure_columns)

for dataset_setting in settings:
    original_data = pd.read_csv(folders['real_dir']+dataset_setting['meta']['filename'])
    
    metadata = dataset_setting['meta']['meta_data']
    
    
    for sd in dataset_setting['meta']['sd_meta_list']:
        synthetic_data = pd.read_csv(sd['path'])
        
        synthetic_data.drop(columns=['Unnamed: 0'], inplace=True)
        
        
        pf_measures = compute_all_pf_measures(original_data=original_data,
                                              synthetic_data=synthetic_data,
                                              metadata=metadata,
                                              SD_id=sd['id'])
        
        display(sd['id'])
        display(pf_measures)
        result_df = pd.concat([result_df, pf_measures], axis=0, ignore_index=True)




'SD0Q1_0'

,DatasetName,pMSE,pMSE_time,s_pMSE,s_pMSE_time,cluster_1,cluster_1_time,BNLogLikelihood,BNLogLikelihood_time,GMLogLikelihood,...,ContinousKLDivergence,ContinousKLDivergence_time,DiscreteKLDivergence,DiscreteKLDivergence_time,KSComplement,KSComplement_time,CSTest,CSTest_time,CrossClassification,CrossClassification_time
0,SD0Q1_0,0.164412,0.024945,151.072299,0.022941,15.331423,0.527681,-0.75236,0.042383,-38.538667,...,0.315956,0.09858,NaN,0.00032,0.691895,0.005644,0.72249,0.00078,0.604588,0.086431


'SD0Q2_0'

,DatasetName,pMSE,pMSE_time,s_pMSE,s_pMSE_time,cluster_1,cluster_1_time,BNLogLikelihood,BNLogLikelihood_time,GMLogLikelihood,...,ContinousKLDivergence,ContinousKLDivergence_time,DiscreteKLDivergence,DiscreteKLDivergence_time,KSComplement,KSComplement_time,CSTest,CSTest_time,CrossClassification,CrossClassification_time
0,SD0Q2_0,0.032756,0.021955,27.933126,0.022788,2.082278,0.495643,-0.741804,0.04166,-28.128305,...,0.650945,0.098153,NaN,0.000348,0.863607,0.005405,0.749257,0.000807,0.604167,0.101591


In [12]:
display(result_df)

,DatasetName,pMSE,s_pMSE,Cluster_1,Cluster_5,Cluster_10,BNLikelihood,BNLogLikelihood,GMLogLikelihood,ContinousKLDivergence,...,s_pMSE_time,cluster_1,cluster_1_time,BNLogLikelihood_time,GMLogLikelihood_time,ContinousKLDivergence_time,DiscreteKLDivergence_time,KSComplement_time,CSTest_time,CrossClassification_time
0,SD0Q1_0,0.164412,151.072299,NaN,NaN,NaN,NaN,-0.752360,-38.538667,0.315956,...,0.022941,15.331423,0.527681,0.042383,3.818502,0.098580,0.000320,0.005644,0.000780,0.086431
1,SD0Q2_0,0.032756,27.933126,NaN,NaN,NaN,NaN,-0.741804,-28.128305,0.650945,...,0.022788,2.082278,0.495643,0.041660,3.834316,0.098153,0.000348,0.005405,0.000807,0.101591


In [13]:
spmse1 = s_pmse(original_data, sd0q1)
spmse2 = s_pmse(original_data, sd0q2)
print(f"S_pMSE: SD0Q1: {spmse1}, SD0Q2: {spmse2}" )

S_pMSE: SD0Q1: (152.7052380323427, 0.023820999999998094), SD0Q2: (23.530050195262135, 0.020177300000000287)


In [ ]:
#display(sd0q1.head())
r1 = pmse(original_data, sd0q1)
r2 = pmse(original_data, sd0q2)
print(f"pMSE: SD0Q1: {r1}, SD0Q2: {r2}" )

In [ ]:
from sdmetrics.reports.single_table import QualityReport, DiagnosticReport
q1Report = QualityReport()
q2Report = QualityReport()

d1Report = DiagnosticReport()
d2Report = DiagnosticReport()
meta = settings[0]['meta']['meta_data']

display("SD0Q1")
# d1Report.generate(original_data, sd0q1, metadata)
q1Report.generate(original_data, sd0q1, meta)
display(q1Report.get_details(property_name='Column Shapes'))

display("SD0Q2")
# d2Report.generate(original_data, sd0q2, metadata)
q2Report.generate(original_data, sd0q2, meta)
display(q2Report.get_details(property_name='Column Shapes'))


In [ ]:
# display("SD0Q1")
# d1fig = d1Report.get_visualization(property_name='Coverage')
# d1fig.show()
# q1fig = q1Report.get_visualization(property_name='Column Shapes')
# q1fig.show()

In [ ]:
# display("SD0Q2")
# d2fig = d2Report.get_visualization(property_name='Coverage')
# d2fig.show()
# q2fig = q2Report.get_visualization(property_name='Column Shapes')
# q2fig.show()

In [ ]:
from sdmetrics.reports import utils

fig = utils.get_column_pair_plot(
    real_data=original_data,
    synthetic_data=sd0q1,
    column_names= ['Glucose', 'Age'],
    metadata=metadata
    
)

fig.show()

Method to find optimal number of clusters for the clustering model using the silhouette analysis method.

In [ ]:
#from kmodes.kprototypes import KPrototypes
#from sklearn.metrics import silhouette_score
#import matplotlib.pyplot as plt
#
#n_clusters = 20
#
#o_data = original_data.copy()
##o_data['S'] = 0
#
#s_data = sd0q1.copy()
##s_data['S'] = 1
#
#
#combined_data = pd.concat([o_data, s_data], axis=0)
#scaled_combined_data = standardize_select_columns(combined_data, [8])
#
#silhouette_scores = []
#
#k_range = range(2, 50)
#
#for k in k_range:
#
#    kproto = KPrototypes(n_clusters=k, init='Cao').fit(scaled_combined_data, categorical=[8])
#    silhouette_scores.append(silhouette_score(scaled_combined_data, kproto.labels_))
#
#fig, ax = plt.subplots()
#ax.plot(k_range, silhouette_scores, 'bx-')
#ax.set_title('Silhouette Score Method')
#ax.set_xlabel('Number of clusters')
#ax.set_ylabel('Silhouette Scores')
#plt.xticks(k_range)
#plt.tight_layout()
#plt.show()

In [ ]:
#import numpy as np
#from sklearn.preprocessing import StandardScaler
## check implementation of cluster analysis
#o_data = original_data.copy()
#o_data['S'] = 0
#
#s_data = sd0q1.copy()
#s_data['S'] = 1
#
#
#combined_data = pd.concat([o_data, s_data], axis=0)
#scaled_combined_data = standardize_select_columns(combined_data, [8])
#
#k=4
#kproto = KPrototypes(n_clusters=k, init='Cao').fit(scaled_combined_data, categorical=[8])
#
#cluster_labels = kproto.labels_
#
#original_data_count = o_data.shape[0]    # number of samples in original data
#synthetic_data_count = s_data.shape[0]  # number of samples in synthetic data
#total_data_count = original_data_count + synthetic_data_count
#
#constant_c = original_data_count / (original_data_count + synthetic_data_count)
#
#display(cluster_labels)
#
#
#scaler = StandardScaler()
#column_indices = np.arange(combined_data.shape[1])
#
#columns_to_standardize = np.setdiff1d(column_indices, [8,9])
#
#combined_data.iloc[:, columns_to_standardize] = scaler.fit_transform(combined_data.iloc[:, columns_to_standardize])
#
##for cluster_id in range(k):
##
##    # TODO: add column, and identify dataset sample from the cluster
##    original_cluster_data_count = np.sum(cluster_labels[:original_data_count] == cluster_id)
##    synthetic_cluster_data_count = np.sum(cluster_labels[original_data_count:] == cluster_id)
##
##    total_cluster_data_count = original_cluster_data_count + synthetic_cluster_data_count